In [1]:
load("../Sage/LegendrePolynomials.sage")
load("../Sage/SymbolicVectorMatrix.sage")

LegendrePolynomials.sage tests passed


In [2]:
num_moments = 2 # includes constant moment
num_eqns = 1 + 2 * num_moments
t = var('t')
x = var('x')
y = var('y')
z = var('z')
g = var('g')
e_x = var('e_x')
e_y = var('e_y')
e_z = var('e_z')
nu = var('nu')
lambda_ = var('lambda_')
i = var('i', domain='integer')
j = var('j', domain='integer')
k = var('k', domain='integer')
h = function('h', nargs=3)(t, x, y)
h_b = function('h_b', nargs=2)(x, y)
u = function('u', nargs=3)(t, x, y)
v = function('v', nargs=3)(t, x, y)
get_symbolic = lambda str_name: function(str_name, nargs=3)(t, x, y)
alpha = get_vector_symbolic('alpha', num_moments, get_symbolic)
beta = get_vector_symbolic('beta', num_moments, get_symbolic)

In [3]:
phi = get_legendre_polynomials_fixed_lower_endpoint(num_moments, 1, 0, 1)
A = [[[(2*i + 1)*(phi[i] * phi[j] * phi[k]).integrate(x, 0, 1) for k in range(num_moments)] for j in range(num_moments)] for i in range(num_moments)]
B = [[[(2 * i + 1) * (phi[i].derivative(x) * (phi[j](x=z)).integrate(z, 0, x) * phi[k]).integrate(x, 0, 1) for k in range(num_moments)] for j in range(num_moments)] for i in range(num_moments)]
C = [[(phi[i].derivative(x) * phi[j].derivative(x)).integrate(x, 0, 1) for j in range(num_moments)] for i in range(num_moments)]
D = [(h * alpha[i]).derivative(x) + (h * beta[i]).derivative(y) for i in range(num_moments)]

# fluxes
fx_list = [
    h * u, 
    h * u^2 + h * sum([alpha[j]^2 / (2 * j + 1) for j in range(1,num_moments)]) + 1/2 * g * e_z * h^2, 
    h * u * v + h * sum([alpha[j] * beta[j] / (2 * j + 1) for j in range(1, num_moments)]),
]
fy_list = [
    h * v,
    h * u * v + h * sum([alpha[j] * beta[j] / (2 * j + 1) for j in range(1, num_moments)]),
    h * v^2 + h * sum([beta[j]^2 / (2 * j + 1) for j in range(1, num_moments)]) + 1/2 * g * e_z * h^2,
]

# source term
p_list = [
    0,
    -nu / lambda_ * (u + sum([alpha[j] for j in range(1, num_moments)])) + h * g * (e_x - e_z * h_b.derivative(x)),
    -nu / lambda_ * (v + sum([beta[j] for j in range(1, num_moments)])) + h * g * (e_y - e_z * h_b.derivative(y)),
]

for i in range(1, num_moments):
    fx_list.append(2 * h * u * alpha[i] + h * sum([sum([A[i][j][k] * alpha[j] * alpha[k] for k in range(1, num_moments)]) for j in range(1, num_moments)]))
    fx_list.append(h * u * beta[i] + h * v * alpha[i] + h * sum([sum([A[i][j][k] * alpha[j] * beta[k] for k in range(1, num_moments)]) for j in range(1, num_moments)]))
    
    fy_list.append(h * u * beta[i] + h * v * alpha[i] + h * sum([sum([A[i][j][k] * alpha[j] * beta[j] for k in range(1, num_moments)]) for j in range(1, num_moments)]))
    fy_list.append(2 * h * v * beta[i] + h * sum([sum([A[i][j][k] * beta[j] * beta[k] for k in range(1, num_moments)]) for j in range(1, num_moments)]))
    
    p_list.append(-(2 * i + 1) * nu / lambda_ * (u + sum([(1 + lambda_ / h * C[i][j]) * alpha[j] for j in range(1, num_moments)])))
    p_list.append(-(2 * i + 1) * nu / lambda_ * (v + sum([(1 + lambda_ / h * C[i][j]) * beta[j] for j in range(1, num_moments)])))

# nonconservative product matrices
gx_lists = [[0 for i in range(num_eqns)] for j in range(num_eqns)]
gy_lists = [[0 for i in range(num_eqns)] for j in range(num_eqns)]

for i in range(1, num_moments):
    a_i_eqn = 1 + 2 * i
    b_i_eqn = 2 + 2 * i
    for j in range(1, num_moments):
        # (h alpha_j)_x term, column index
        a_j_eqn = 1 + 2 * j
        # (h beta_j)_y term, column index
        b_j_eqn = 2 + 2 * j
        
        # h alpha[i] equation
        # (h alpha_i)_t + ... = ... - sum{j=1}{N}{D_j \sum{k=1}{N}{B_ijk alpha_k}} + ...
        # (h alpha_j)_x sum{k=1}{N}{-B_ijk alpha_k}
        gx_lists[a_i_eqn][a_j_eqn] = sum([-B[i][j][k] * alpha[k] for k in range(1, num_moments)])
        # (h beta_j)_y sum{k=1}{N}{-B_ijk alpha_k}
        gy_lists[a_i_eqn][b_j_eqn] = sum([-B[i][j][k] * alpha[k] for k in range(1, num_moments)]) 
        
        # (h beta_i)_t + ... = ... - sum{j=1}{N}{D_j \sum{k=1}{N}{B_ijk beta_k}} + ...
        # (h alpha_j)_x sum{k=1}{N}{-B_ijk beta_k}
        gx_lists[b_i_eqn][a_j_eqn] = sum([-B[i][j][k] * beta[k] for k in range(1, num_moments)])
        # (h beta_j)_y sum{k=1}{N}{-B_ijk beta_k}
        gy_lists[b_i_eqn][b_j_eqn] = sum([-B[i][j][k] * beta[k] for k in range(1, num_moments)]) 
        
    
    # (h alpha_i)_t + ... = u D_i + ...
    # u D_i = u (h alpha_i)_x + u (h beta_i)_y
    gx_lists[a_i_eqn][a_i_eqn] += u
    gy_lists[a_i_eqn][b_i_eqn] += u
    
    # (h beta_i)_t + ... = v D_i + ... 
    # v D_i = v (h alpha_i)_x + v (h beta_i)_y
    gx_lists[b_i_eqn][a_i_eqn] += v
    gy_lists[b_i_eqn][b_i_eqn] += v

fx = matrix(fx_list).transpose()
fy = matrix(fy_list).transpose()

gx = matrix(gx_lists)
gy = matrix(gy_lists)

p = matrix(p_list).transpose()

[                                                               h(t, x, y)*u(t, x, y)]
[1/2*e_z*g*h(t, x, y)^2 + 1/3*alpha_1(t, x, y)^2*h(t, x, y) + h(t, x, y)*u(t, x, y)^2]
[  1/3*alpha_1(t, x, y)*beta_1(t, x, y)*h(t, x, y) + h(t, x, y)*u(t, x, y)*v(t, x, y)]
[                                            2*alpha_1(t, x, y)*h(t, x, y)*u(t, x, y)]
[      beta_1(t, x, y)*h(t, x, y)*u(t, x, y) + alpha_1(t, x, y)*h(t, x, y)*v(t, x, y)]

In [4]:
print(latex(fx))
print(latex(fy))
print(latex(gx))
print(latex(gy))
print(latex(p))

\left(\begin{array}{r}
h\left(t, x, y\right) u\left(t, x, y\right) \\
\frac{1}{2} \, e_{z} g h\left(t, x, y\right)^{2} + \frac{1}{3} \, \alpha_{1}\left(t, x, y\right)^{2} h\left(t, x, y\right) + h\left(t, x, y\right) u\left(t, x, y\right)^{2} \\
\frac{1}{3} \, \alpha_{1}\left(t, x, y\right) \beta_{1}\left(t, x, y\right) h\left(t, x, y\right) + h\left(t, x, y\right) u\left(t, x, y\right) v\left(t, x, y\right) \\
2 \, \alpha_{1}\left(t, x, y\right) h\left(t, x, y\right) u\left(t, x, y\right) \\
\beta_{1}\left(t, x, y\right) h\left(t, x, y\right) u\left(t, x, y\right) + \alpha_{1}\left(t, x, y\right) h\left(t, x, y\right) v\left(t, x, y\right)
\end{array}\right)
\left(\begin{array}{r}
h\left(t, x, y\right) v\left(t, x, y\right) \\
\frac{1}{3} \, \alpha_{1}\left(t, x, y\right) \beta_{1}\left(t, x, y\right) h\left(t, x, y\right) + h\left(t, x, y\right) u\left(t, x, y\right) v\left(t, x, y\right) \\
\frac{1}{2} \, e_{z} g h\left(t, x, y\right)^{2} + \frac{1}{3} \, \beta_{1}\left(t, x, y\righ

In [137]:
#oned_dict_ = {v: 0, beta_0: 0, beta_1: 0, beta_2: 0}
oned_dict_ = {v: 0}
for i in range(num_moments):
    oned_dict_[beta[i]] = 0
print(latex(fx.subs(oned_dict_)))
#print(latex(fy.subs(oned_dict_)))
print(latex(gx.subs(oned_dict_)))
#print(latex(gy.subs(oned_dict_)))
print(latex(p.subs(oned_dict_)))

\left(\begin{array}{r}
h\left(t, x, y\right) u\left(t, x, y\right) \\
\frac{1}{2} \, e_{z} g h\left(t, x, y\right)^{2} + h\left(t, x, y\right) u\left(t, x, y\right)^{2} + \frac{1}{105} \, {\left(35 \, \alpha_{1}\left(t, x, y\right)^{2} + 21 \, \alpha_{2}\left(t, x, y\right)^{2} + 15 \, \alpha_{3}\left(t, x, y\right)^{2}\right)} h\left(t, x, y\right) \\
0 \\
2 \, \alpha_{1}\left(t, x, y\right) h\left(t, x, y\right) u\left(t, x, y\right) + \frac{2}{35} \, {\left(14 \, \alpha_{1}\left(t, x, y\right) \alpha_{2}\left(t, x, y\right) + 9 \, \alpha_{2}\left(t, x, y\right) \alpha_{3}\left(t, x, y\right)\right)} h\left(t, x, y\right) \\
0 \\
2 \, \alpha_{2}\left(t, x, y\right) h\left(t, x, y\right) u\left(t, x, y\right) + \frac{2}{21} \, {\left(7 \, \alpha_{1}\left(t, x, y\right)^{2} + 3 \, \alpha_{2}\left(t, x, y\right)^{2} + 9 \, \alpha_{1}\left(t, x, y\right) \alpha_{3}\left(t, x, y\right) + 2 \, \alpha_{3}\left(t, x, y\right)^{2}\right)} h\left(t, x, y\right) \\
0 \\
2 \, \alpha_{3}\left(t, 

In [19]:
phi0(x) = 1
phi1(x) = 1 - 2 * x
phi2(x) = 1 - 6 * x + 6 * x^2
phi3(x) = 1 - 12 * x + 30 * x^2 - 20 * x^3

In [30]:
(phi2*phi2).integrate(x, 0, 1)

1/5

In [31]:
phi0(x) = 1
phi1(x) = 1 + x
phi2(x) = 1 - 6 * x - 3 * x^2

In [32]:
(phi0*phi1).integrate(x, 0, 1)

3/2